In [1]:
import pandas as pd
import json
import numpy as np

In [54]:
# Read results
autoencoder_name = 'train_autoencoder_1024_deep3_users_projects_0.8_5'
results = pd.read_json('../../data/experiment-results/cf-autoencoder/%s.json' % (autoencoder_name), lines=True)

results = results.T
results['json'] = results[0]

newDataframe = pd.DataFrame(columns=['user_index', 'precision', 'recall', 'y_pred', 'y_true', 'len_yPred', 'len_yTrue', 'avg_precision', 'rmse'])
newDataframe['y_pred'] = newDataframe['y_pred'].astype(object)
newDataframe['y_true'] = newDataframe['y_true'].astype(object)

for index, row in results.iterrows():
    newSeries = pd.Series(index=['user_index', 'precision', 'recall', 'y_pred', 'y_true', 'len_yPred', 'len_yTrue'])
    newSeries['y_pred'] = newSeries['y_pred'].astype(object)
    newSeries['y_true'] = newSeries['y_true'].astype(object)

    newSeries['user_index'] = row['json']['user_index']
    newSeries['precision'] = row['json']['precision']
    newSeries['recall'] = row['json']['recall']

    things1 = np.array(row['json']['y_pred']).astype('str')
    things2 = np.array(row['json']['y_true']).astype('str')
    y_pred_string = '[' + ', '.join(things1) + ']'
    y_true_string = '[' + ', '.join(things2) + ']'
    
    newSeries['y_pred'] = y_pred_string
    newSeries['y_true'] = y_true_string
    
    newSeries['len_yPred'] = len(row['json']['y_pred'])
    newSeries['len_yTrue'] = len(row['json']['y_true'])
    
    newSeries['rmse'] = row['json']['rmse']
    newSeries['avg_precision'] = row['json']['avg_precision']
    
    newDataframe.loc[index] = newSeries
    
newDataframe.describe()

,user_index,precision,recall,avg_precision,rmse
count,1032.000000,1032.000000,1032.000000,1032.000000,1.032000e+03
mean,515.500000,0.000775,0.000651,0.004041,5.757665e-02
std,298.057042,0.012433,0.012541,0.006591,2.218319e-02
min,0.000000,0.000000,0.000000,0.000000,7.824764e-28
25%,257.750000,0.000000,0.000000,0.001959,4.425905e-02
50%,515.500000,0.000000,0.000000,0.002938,5.420605e-02
75%,773.250000,0.000000,0.000000,0.003918,6.259175e-02
max,1031.000000,0.200000,0.333333,0.168626,2.278374e-01


In [55]:
all_preds = np.array([])
for i in range(0, newDataframe.shape[1]):
    preds = np.asarray(newDataframe.iloc[i]['y_pred'][1:-1].split(', '), dtype=int)
    all_preds = np.append(all_preds, preds)
    
len(set(all_preds))

5

In [56]:
set(all_preds)

{1016.0, 1017.0, 1018.0, 1019.0, 1020.0}

In [63]:
newDataframe[newDataframe['precision'] == 0.2]

,user_index,precision,recall,y_pred,y_true,len_yPred,len_yTrue,avg_precision,rmse
738,738.0,0.2,0.200000,"[1016, 1017, 1018, 1019, 1020]","[19, 56, 402, 880, 1018]",5,5,0.043918,0.069980
779,779.0,0.2,0.038462,"[1016, 1017, 1018, 1019, 1020]","[21, 115, 126, 190, 245, 316, 324, 337, 345, 3...",5,26,0.043717,0.159578
854,854.0,0.2,0.333333,"[1016, 1017, 1018, 1019, 1020]","[21, 816, 1019]",5,3,0.168626,0.054206
876,876.0,0.2,0.100000,"[1016, 1017, 1018, 1019, 1020]","[24, 56, 402, 714, 754, 782, 832, 880, 1006, 1...",5,10,0.028815,0.098966


In [58]:
# Load projects
project_ids = pd.read_pickle('../../data/processed/cf_projects.pkl')
projects = pd.read_pickle('../../data/processed/project_data')

In [64]:
# Projects that were done
projects[projects['project_id'].isin(project_ids.iloc[[19, 56, 402, 880, 1018]].values.flatten())]

,UN_regions,country,description,error,guid,origin,regions,tags,title,topics,url,project_id
23,[],None,Project Squirrel is calling all citizen scient...,NaN,3c8f4f60-f30d-5ca3-bec6-c73db6217260,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[girl scouts, internal_girl_scouts, squirrel, ...",Project Squirrel,"[Nature & Outdoors, Animals, Ecology & Environ...",https://scistarter.com/project/38-Project-Squi...,38
80,[],None,Seven out of 10 people in the US have never se...,NaN,083ad715-4820-5958-a43b-3a32c3ae4532,scistarter,[],"[girl scouts, internal_girl_scouts, light poll...",Globe at Night,"[Archeology & Cultural, Agriculture, Events, C...",https://scistarter.com/project/169-Globe-at-Ni...,169
664,[],None,Help scientists analyze movies from the brain ...,NaN,4442a7ee-6c54-5a55-9ec5-ae4b9b52008e,scistarter,[],"[alz, alzheimer, alzheimer's disease, biomedic...","Stall Catchers, by EyesOnALZ","[Health & Medicine, Chemistry, Biology]",https://scistarter.com/project/1014-Stall-Catc...,1014
1408,[],None,OpenLitterMap.com is a web-based litter mappin...,NaN,c6e326c4-a259-5a5b-a3f4-7fcfee669f3a,scistarter,[],"[blockchain, ethereum, litter, littercoin, mar...",OpenLitterMap.com,"[Geography, Events, Crowd Funding, Ecology & E...",https://scistarter.com/project/18681-OpenLitte...,18681
1745,[],None,STOP! The State of Plastic Pollution- is the i...,NaN,4c94b175-8363-5148-a607-766ec2d5a336,scistarter,[],[],STOP! The State of Plastic Pollution #1,[],https://scistarter.com/project/20591-STOP-The-...,20591


In [65]:
# Projects that were recommended
# 585, 586, 587, 588, 589
projects[projects['project_id'].isin(project_ids.iloc[[1016, 1017, 1018, 1019, 1020]].values.flatten())]

,UN_regions,country,description,error,guid,origin,regions,tags,title,topics,url,project_id
1743,[],None,Bring technology in the classroom while engagi...,NaN,9ff9fc6c-c2a2-5159-a76c-56f8b8170cc1,scistarter,[],[],Free Webinar - Technology in the Classroom wit...,[],https://scistarter.com/project/20573-Free-Webi...,20573
1744,[],None,Investigate and explore possible associations ...,NaN,91ce61e4-c8d3-5d9d-952f-c89beb616c4f,scistarter,[],"[cardiac disease, dcm, dilated cardiomyopathy,...",Doberman Diversity Project,"[Biology, Health & Medicine, Animals]",https://scistarter.com/project/20577-Doberman-...,20577
1745,[],None,STOP! The State of Plastic Pollution- is the i...,NaN,4c94b175-8363-5148-a607-766ec2d5a336,scistarter,[],[],STOP! The State of Plastic Pollution #1,[],https://scistarter.com/project/20591-STOP-The-...,20591
1746,[],None,ABOUT US: iCivics was founded by Justice Sandr...,NaN,217a1657-f779-5050-a8a9-fb10739ef525,scistarter,[],"[civics, constitution, educational games, fami...",Democracy At Play - Constitution Day Challenge,"[Education, Social Science]",https://scistarter.com/project/20603-Democracy...,20603
1749,[],None,The CrowdWater game is based on data from the ...,NaN,c3a1ecee-5e47-5866-96f3-7ca9cd707307,scistarter,[],"[crowdwater, drought, flood, game, geography, ...",CrowdWater game,"[Ocean, Water, Marine & Terrestrial, Ecology &...",https://scistarter.com/project/20650-CrowdWate...,20650
